# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The api, however, provides you with a lot of information that will not be pertinent to your analysis. YOu will pull data from the api and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the api will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

___

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

- Write a function to make a call to the yelp API

In [435]:
import pandas as pd
import requests
import json
import os.path
from os import path
import csv

In [436]:
yelp_id = "4eucNo6rT7PNhkEzgMEKJQ"
yelp_key = "7bG0ESSZL3r562AjGaZ-C2NFWWlOuwXiMJPb_xI0mUiziEKXWS9DVhiM_t4Jk0xdqLHc1qsKtDqdhrPRut4sHlmFac3p0rKuBKm3vUhAW0Vhb33NFzVaAaE7nXOaYHYx"

In [437]:
def yelp_call(url_params, api_key):
    headers = {'Authorization': 'Bearer ' + api_key}
    response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=url_params)
    data = response.json()
    return data

- Write a function to parse the API response so that you can easily insert the data in to the DB

In [438]:
url_params = {
    'term': 'restaurant',
    'location': 'Long+Beach+Island+NJ',
    'limit': 10,
    'offset': 0}
api_key = yelp_key
data = yelp_call(url_params, api_key)

In [463]:
all_categories = ['id', 'name', 'is_closed', 'url', 'review_count', 'categories', 
                  'rating', 'latitude', 'longitude', 'address1', 'address2', 
                  'address3', 'city', 'state', 'zip_code', 'price']

In [464]:
def parse_results(results):
    parsed_data=[]
    for business in results['businesses']:
        if 'price' not in business:
            business['price'] = None
        business_data = [business['id'], business['name'], business['is_closed'], business['url'], 
                         business['review_count'], business['categories'], business['rating'], 
                         business['coordinates']['latitude'], business['coordinates']['longitude'], 
                         business['location']['address1'], business['location']['address2'], 
                         business['location']['address3'], business['location']['city'], 
                         business['location']['state'], business['location']['zip_code'], business['price']]
        business_data_zip = dict(zip(all_categories, business_data))
        parsed_data.append(business_data_zip)
    return parsed_data

- Write a function to take your parsed data and add it to the csv file where you will store all of your results. 

In [441]:
def df_save(csv_filepath, parsed_results):
    parsed_df = pd.DataFrame(parsed_results)
    if path.exists(csv_filepath) == False:
        parsed_df.to_csv(csv_filepath, header=True)
    else:
        with open(csv_filepath, 'a') as f:
            parsed_df.to_csv(f, header=False)

- Write a script that combines the three functions above into a single process.

In [42]:
pip install helper-funcs

  Created wheel for helper-funcs: filename=helper_funcs-0.1.35-py3-none-any.whl size=5991 sha256=bb4bde350ddfe0f739ec5349fd066c0d923ff944636bb91d8604e13fdb590c7a
  Stored in directory: /Users/jameshodgens/Library/Caches/pip/wheels/c9/4d/0b/0e48fddaeb3676093c6c24dd72e5dd3fd901faad19d9deae42
Successfully built helper-funcs
Note: you may need to restart the kernel to use updated packages.


While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [43]:
from helper_funcs import *

In [442]:
url_params = {
    'term': 'restaurant',
    'location': 'Long+Beach+Island+NJ',
    'limit': 50,
    'offset': 0}
api_key = yelp_key
data = yelp_call(url_params, api_key)

In [465]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < 200:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save('yelp_businesses.csv', parsed_results)
    
    #increment the counter by 50 to move on to the next results
    cur += 50

___

In [445]:
business_df = pd.read_csv('yelp_businesses.csv')

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you ahve a model of the functions you will need to write, and how to pull them together in one script. For this part, you ahve the process below 

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

In [446]:
business_ids=business_df['id']
business_ids

0      YVERvPBMurED_1yWtigJQw
1      uMNgVdpOI22QFTkxaVo9sA
2      uBMQXMgvK1iVdmS4CglCKA
3      QKhSd92X3EF8fComzbA45w
4      fstH18hKRdg5LLtocV8G_A
                ...          
195    GaPMgtqB7VOtTy_22IVaPA
196    pjngrCUGboFoN_ECLV4DnQ
197    zdDGGZlW937ACVmbedGrBA
198    78UO-Qgt-Ja0RFIRHWxPDg
199    _tTGWLMYPHBh2iCeOmscQg
Name: id, Length: 200, dtype: object

- Write a function that takes a business id and makes a call to the API for reivews


In [447]:
#Test code before building function
business_id=business_df['id'][0]
url = 'https://api.yelp.com/v3/businesses/'+business_id+'/reviews'
headers = {
    'Authorization': 'Bearer ' + creds['key']
}

response = requests.get(url, headers=headers)
reviews_test = response.json()
reviews_test

{'reviews': [{'id': 'LsdV0ahKp0-u1_yanCfNcQ',
   'url': 'https://www.yelp.com/biz/salt-kitchen-and-bar-ship-bottom?adjust_creative=4eucNo6rT7PNhkEzgMEKJQ&hrid=LsdV0ahKp0-u1_yanCfNcQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=4eucNo6rT7PNhkEzgMEKJQ',
   'text': 'We are staying at the Hotel LBI for a Covid staycation and rather than subjecting other diners to my 3 rambunctious kids we opted to order delivery. The...',
   'rating': 4,
   'time_created': '2021-02-28 19:02:13',
   'user': {'id': 'FhNdhb9GhwzM50JCMM7JhQ',
    'profile_url': 'https://www.yelp.com/user_details?userid=FhNdhb9GhwzM50JCMM7JhQ',
    'image_url': None,
    'name': 'Stephanie C.'}},
  {'id': 'NxP4ajjpYMNsJR9QsU_z-A',
   'url': 'https://www.yelp.com/biz/salt-kitchen-and-bar-ship-bottom?adjust_creative=4eucNo6rT7PNhkEzgMEKJQ&hrid=NxP4ajjpYMNsJR9QsU_z-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=4eucNo6rT7PNhkEzgMEKJQ',
   'text': 'Celebrated my birthday here and 

In [455]:
def review_call(business_id):
    headers = {'Authorization': 'Bearer ' + api_key}
    response = requests.get('https://api.yelp.com/v3/businesses/'+business_id+'/reviews', headers=headers)
    data = response.json()
    return data

- Write a function to parse out the relevant information from the reviews

In [459]:
def parse_reviews(result, business_id):
    parsed_results = []
    for review in result['reviews']:
        review_info = {'review_id': review['id'], 'text': review['text'], 'rating': review['rating'], 
                           'business_id': business_id}
        parsed_results.append(review_info)
    return parsed_results

- Write a function to save the parse data into a csv file containing all of the reviews. 

In [461]:
def df_save(csv_filepath, parsed_results):
    parsed_df = pd.DataFrame(parsed_results)
    if path.exists(csv_filepath) == False:
        parsed_df.to_csv(csv_filepath, header=True)
    else:
        with open(csv_filepath, 'a') as f:
            parsed_df.to_csv(f, header=False)

- Combine the functions above into a single script  

In [462]:
for business_id in business_ids:
    data = review_call(business_id)
    parsed_data = parse_reviews(data, business_id)
    df_save('yelp_reviews.csv', parsed_data)

___

## Part 4 -  Using python and pandas, write code to answer the questions below. 


- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


In [347]:
business_df

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0,QKhSd92X3EF8fComzbA45w,the-chicken-or-the-egg-beach-haven,The Chicken or the Egg,https://s3-media4.fl.yelpcdn.com/bphoto/VoIvIJ...,False,https://www.yelp.com/biz/the-chicken-or-the-eg...,912,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 39.5646712, 'longitude': -74.2389...",['delivery'],$$,"{'address1': '207 N Bay Ave', 'address2': '', ...",+16094923695,(609) 492-3695,8752.19418706395
1,1,YVERvPBMurED_1yWtigJQw,salt-kitchen-and-bar-ship-bottom,Salt Kitchen & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/hMih4j...,False,https://www.yelp.com/biz/salt-kitchen-and-bar-...,72,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 39.6527240135516, 'longitude': -7...",['delivery'],NaN,"{'address1': '350 W 8th St', 'address2': '', '...",+16099918001,(609) 991-8001,2159.726113306633
2,2,uMNgVdpOI22QFTkxaVo9sA,the-beach-house-restaurant-beach-haven-terrace,The Beach House Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/P3CUfh...,False,https://www.yelp.com/biz/the-beach-house-resta...,101,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.585813, 'longitude': -74.223744}","['restaurant_reservation', 'delivery']",$$$,"{'address1': '13015 Long Beach Blvd', 'address...",+16094921997,(609) 492-1997,6067.463933556038
3,3,fstH18hKRdg5LLtocV8G_A,the-arlington-ship-bottom,The Arlington,https://s3-media1.fl.yelpcdn.com/bphoto/h_adK4...,False,https://www.yelp.com/biz/the-arlington-ship-bo...,344,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 39.6464704060192, 'longitude': -7...",['delivery'],$$$,"{'address1': '1302 Long Beach Blvd', 'address2...",+16094948848,(609) 494-8848,1827.4537299185029
4,4,uBMQXMgvK1iVdmS4CglCKA,ship-bottom-shellfish-ship-bottom,Ship Bottom Shellfish,https://s3-media1.fl.yelpcdn.com/bphoto/z0nphd...,False,https://www.yelp.com/biz/ship-bottom-shellfish...,294,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 39.6443336, 'longitude': -74.1802...",['delivery'],$$,"{'address1': '1721 Long Beach Blvd', 'address2...",+16094940088,(609) 494-0088,1519.4403171809995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,45,0LL-pGMFeh4gIUGlO0wbAg,philly-pretzel-factory-beach-haven,Philly Pretzel Factory,NaN,False,https://www.yelp.com/biz/philly-pretzel-factor...,1,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",1.0,"{'latitude': 39.57037495375017, 'longitude': -...",[],$$$,"{'address1': '325 9th St', 'address2': '', 'ad...",+16094923245,(609) 492-3245,8113.562915932857
196,46,GaPMgtqB7VOtTy_22IVaPA,philly-pretzel-factory-stafford-township,Philly Pretzel Factory,NaN,False,https://www.yelp.com/biz/philly-pretzel-factor...,7,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",2.5,"{'latitude': 39.6893069, 'longitude': -74.2450...",['delivery'],$,"{'address1': '412 East Bay Ave', 'address2': '...",+16094890009,(609) 489-0009,7469.773835884276
197,47,zdDGGZlW937ACVmbedGrBA,golden-jade-manahawkin,Golden Jade,https://s3-media2.fl.yelpcdn.com/bphoto/shzqXq...,False,https://www.yelp.com/biz/golden-jade-manahawki...,22,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.0,"{'latitude': 39.706691, 'longitude': -74.276422}",[],$$,"{'address1': '601 Washington Ave', 'address2':...",+16099788168,(609) 978-8168,10667.37260747689
198,48,78UO-Qgt-Ja0RFIRHWxPDg,pro-shop-beach-haven,Pro Shop,NaN,False,https://www.yelp.com/biz/pro-shop-beach-haven?...,1,"[{'alias': 'delis', 'title': 'Delis'}]",1.0,"{'latitude': 39.595861, 'longitude': -74.216481}",[],NaN,"{'address1': '11106 Long Beach Blvd', 'address...",+16094920515,(609) 492-0515,4792.0570773124255


In [348]:
#1 - five most reviewed businesses
business_df.sort_values('review_count', ascending = False)[0:5]

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0,QKhSd92X3EF8fComzbA45w,the-chicken-or-the-egg-beach-haven,The Chicken or the Egg,https://s3-media4.fl.yelpcdn.com/bphoto/VoIvIJ...,False,https://www.yelp.com/biz/the-chicken-or-the-eg...,912,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 39.5646712, 'longitude': -74.2389...",['delivery'],$$,"{'address1': '207 N Bay Ave', 'address2': '', ...",+16094923695,(609) 492-3695,8752.19418706395
18,18,WCTJFwg5vPqwwzo-21v8pQ,the-old-causeway-steak-and-oyster-house-staffo...,The Old Causeway Steak & Oyster House,https://s3-media1.fl.yelpcdn.com/bphoto/0iTWJT...,False,https://www.yelp.com/biz/the-old-causeway-stea...,389,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.6739616394043, 'longitude': -7...",['delivery'],$$,"{'address1': '1201 E Bay Ave', 'address2': '',...",+16094881327,(609) 488-1327,4914.269174321862
30,30,JrukXlFWLBEkWJSR1pcBPQ,mud-city-crab-house-manahawkin,Mud City Crab House,https://s3-media2.fl.yelpcdn.com/bphoto/lWczvn...,False,https://www.yelp.com/biz/mud-city-crab-house-m...,384,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 39.6745693227362, 'longitude': -7...",['delivery'],$$,"{'address1': '1185 E Bay Ave', 'address2': '',...",+16099783660,(609) 978-3660,4982.353775066047
3,3,fstH18hKRdg5LLtocV8G_A,the-arlington-ship-bottom,The Arlington,https://s3-media1.fl.yelpcdn.com/bphoto/h_adK4...,False,https://www.yelp.com/biz/the-arlington-ship-bo...,344,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 39.6464704060192, 'longitude': -7...",['delivery'],$$$,"{'address1': '1302 Long Beach Blvd', 'address2...",+16094948848,(609) 494-8848,1827.4537299185029
52,2,U7y3ih-GofVs8fUWf_qAgA,buckalews-restaurant-and-tavern-beach-haven,Buckalew's Restaurant & Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/HlTNGb...,False,https://www.yelp.com/biz/buckalews-restaurant-...,339,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 39.56375, 'longitude': -74.23939}",['delivery'],$$,"{'address1': '101 N Bay Ave', 'address2': '', ...",+16094921065,(609) 492-1065,8860.291246232726


In [349]:
#2 - highest rating
business_df.groupby('rating').count().sort_values('rating', ascending = False)['id']

rating
5.0    13
4.5    25
4.0    62
3.5    52
3.0    28
2.5    10
2.0     6
1.5     2
1.0     2
Name: id, dtype: int64

In [350]:
#3 - percentage greater than or qeual to 4.5
rating_counts = business_df.groupby('rating').count().sort_values('rating', ascending = False)['id']
best_restaurants = rating_counts[5.0]+rating_counts[4.5]
total_restaurants = business_df.shape[0]
best_restaurants/total_restaurants

0.19

In [351]:
#4 - percentage less than 3
worst_restaurants = rating_counts[1.0] + rating_counts[1.5] + rating_counts[2.0] + rating_counts[2.5]
worst_restaurants/total_restaurants

0.1

In [352]:
#5 - percentage by dollar sign
business_df.groupby('price').count()['id']

price
$      43
$$     90
$$$    18
Name: id, dtype: int64

In [353]:
print('Restaurants with one dollar sign: ' + str(businessdf.groupby('price').count()['id']['$']/total_restaurants))
print('Restaurants with two dollar signs: ' + str(businessdf.groupby('price').count()['id']['$$']/total_restaurants))
print('Restaurants with three dollar signs: ' + str(businessdf.groupby('price').count()['id']['$$$']/total_restaurants))
print('Restaurants with no dollar signs: ' + str(1-(businessdf.groupby('price').count()['id']['$'] + businessdf.groupby('price').count()['id']['$$'] + businessdf.groupby('price').count()['id']['$$$'])/total_restaurants))

Restaurants with one dollar sign: 0.015
Restaurants with two dollar signs: 0.145
Restaurants with three dollar signs: 0.055
Restaurants with no dollar signs: 0.785


In [354]:
reviewdf = pd.read_csv('yelp_reviews.csv')

In [359]:
#6 - most reviewed business
most_reviewed = businessdf.sort_values('review_count', ascending = False)[0:1]['id'][0]
most_reviewed

'QKhSd92X3EF8fComzbA45w'

In [361]:
# get reviews for most reviewed business
reviewdf[reviewdf['business_id'] == most_reviewed]['text']

0    They are the best wings I have had. There whit...
1    This really is the spot if you are from the is...
2    This LBI staple just never disappoints. We wer...
Name: text, dtype: object

In [225]:
#7 - text of the most recent review for 5.0 rating with the most reviews
five_point_restaurant = businessdf[businessdf['rating'] == 5.0]['id'][8]
reviewdf[reviewdf['business id'] == five_point_restaurant]['text'][18]

'I have to give it to this place, it does not look like much sitting inside Fantasy Island Amusement Park in Beach Haven LBI, however with the expanded...'

In [379]:
#8 - text of the most recent review for lowest rated business with the least reviews
one_star = businessdf[businessdf['rating'] == 1.0].sort_values('review_count')['id'][0:1][195]
reviewdf[reviewdf['business_id'] == one_star]['text'][556]

'I returned again to this location this week hoping for a good Philly Soft Pretzel.  Well even though this is a Philly Pretzel location, they no longer serve...'

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20

In [391]:
test = reviewdf['user'][0]

In [394]:
eval(test)['name']

'Edward G.'

In [397]:
for user in reviewdf['user']:
    if eval(user)['name'] == 'Paul H.':
        print("He is in here!")
    else:
        continue

In [398]:
reviewdf

,Unnamed: 0,id,url,text,rating,time_created,user,business_id
0,0,lpd_Tf4ylaMyB41P3Wb8JA,https://www.yelp.com/biz/the-chicken-or-the-eg...,They are the best wings I have had. There whit...,5,2021-03-21 13:40:05,"{'id': 'RKWiAZIvnDj2RajMC9Ddxw', 'profile_url'...",QKhSd92X3EF8fComzbA45w
1,1,lcqitQLElJlVRMxdaKcyIQ,https://www.yelp.com/biz/the-chicken-or-the-eg...,This really is the spot if you are from the is...,5,2021-04-09 09:41:54,"{'id': 'ntlAoryIWOPmp9-qaAKYLQ', 'profile_url'...",QKhSd92X3EF8fComzbA45w
2,2,KQXuf6uBBczB1qCJvOu21Q,https://www.yelp.com/biz/the-chicken-or-the-eg...,This LBI staple just never disappoints. We wer...,5,2021-01-01 12:04:01,"{'id': 'mSaD4g37jCy4-95pB6NWKA', 'profile_url'...",QKhSd92X3EF8fComzbA45w
3,0,LsdV0ahKp0-u1_yanCfNcQ,https://www.yelp.com/biz/salt-kitchen-and-bar-...,We are staying at the Hotel LBI for a Covid st...,4,2021-02-28 19:02:13,"{'id': 'FhNdhb9GhwzM50JCMM7JhQ', 'profile_url'...",YVERvPBMurED_1yWtigJQw
4,1,NxP4ajjpYMNsJR9QsU_z-A,https://www.yelp.com/biz/salt-kitchen-and-bar-...,Celebrated my birthday here and it was lovely....,4,2020-09-02 12:42:22,"{'id': 'RG4Irpi84nvnGgyyFecIzA', 'profile_url'...",YVERvPBMurED_1yWtigJQw
...,...,...,...,...,...,...,...,...
562,2,MohE6vGlSiD2C6fMPd7SRg,https://www.yelp.com/biz/golden-jade-manahawki...,"We go here frequently, but sadly (on Mother's ...",1,2019-05-12 18:32:48,"{'id': 'Jwci4ffGYpCLDRc4pxlvCg', 'profile_url'...",zdDGGZlW937ACVmbedGrBA
563,0,TmoEGdkp-c3AB_xi7BRLhA,https://www.yelp.com/biz/pro-shop-beach-haven?...,"Poor service, way overpriced. Children running...",1,2013-08-15 05:53:29,"{'id': 'ne5qhKTEzD20Nf9aG6ZCNg', 'profile_url'...",78UO-Qgt-Ja0RFIRHWxPDg
564,0,o_a4tf7HrHEfaT9OCl9YDw,https://www.yelp.com/biz/bella-christinas-trat...,I had really high hopes for this place. Seein...,2,2013-05-28 13:26:42,"{'id': 'cnlCi0XAYMp-2r4M5Ec9Xw', 'profile_url'...",_tTGWLMYPHBh2iCeOmscQg
565,1,Sec1LP6Wn2GFYD8wVZnURg,https://www.yelp.com/biz/bella-christinas-trat...,All was good.. Equal or better than other Ital...,5,2014-11-24 17:56:45,"{'id': '7-sKlu70SZGlD60Z7dPixg', 'profile_url'...",_tTGWLMYPHBh2iCeOmscQg
